In [ ]:
import numpy as np
import cv2

def position_velocity_sphere(cap, radius, focal_length, fps, upper_color, lower_color):
    
    '''
    
    :param cap: cv2.VideoCapture object
    :param radius: real radius of the sphere in cm
    :param focal_length: focal length of the camera in pixels
    :param fps: frames per second of the camera
    :param upper_color: upper color of the sphere in HSV
    :param lower_color: lower color of the sphere in HSV
    
    :return: velocities, times and position lists
    
    '''
    
    previous_distance_from_screen = 0
    previous_center_distance = 0
    previous_center_x = 0
    previous_center_y = 0
    
    positions = []
    velocities = []
    times = []
    count_time = 0

    while(True):
        ret, frame = cap.read()
        width = int(cap.get(3))
        height = int(cap.get(4))
        center_screen = [int(width/2), int(height/2)]
        
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        mask = cv2.inRange(hsv, lower_green_and_yellow, upper_green_and_yellow)

        countours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        if len(countours) > 0:
            for contour in countours:
                area = cv2.contourArea(contour)
                if area > 1000:

                    # center and radius detection
                    (center_x, center_y), pixel_radius = cv2.minEnclosingCircle(contour)
                    center_x = int(center_x)
                    center_y = int(center_y)
                    radius = int(radius)

                    pixel_per_cm = pixel_radius / radius

                    distance_y = (radius * focal_length) / pixel_radius # distance in cm, that's an accurate way to calculate the distance from the screen
                    position_xz_pix = np.sqrt((center_x - center_screen[0])**2 + (center_y - center_screen[1])**2) # position in pixels
                    distance_xz_pix = np.sqrt((center_x - previous_center_x)**2 + (center_y - previous_center_y)**2) # distance in pixels
                    position_xz = position_xz_pix / pixel_per_cm # distance in cm
                    distance_xz = distance_xz_pix / pixel_per_cm # distance in cm

                    distance_from_prev_y = np.abs(distance_y - previous_distance_from_screen)
                    distance_from_prev_xz = np.abs(distance_xz - previous_center_distance)

                    distance = np.sqrt(distance_from_prev_y**2 + distance_from_prev_xz**2)
                    velocity = distance / (1/fps) / 100 # velocity in m/s
                    velocities.append(velocity)
                    count_time += 1/fps
                    times.append(count_time)
                    positions.append([center_x/pixel_per_cm, center_y/pixel_per_cm])

                    cv2.circle(frame, (center_x, center_y), radius, (0, 255, 0), 1)

                    previous_center_x = center_x
                    previous_center_y = center_y
                    previous_distance_from_screen = distance_y
                    previous_center_distance = distance_xz
                    text1 = f'Position: {position_xz:.2f} cm'
                    text2 = f'Distance screen: {distance_y:.2f} cm'
                    text3 = f'Velocity: {velocity:.2f} m/s'
                    cv2.putText(frame, text1, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(frame, text2, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.putText(frame, text3, (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        mask = cv2.bitwise_and(frame, frame, mask=mask)

        cv2.imshow('Mask', mask)
        cv2.imshow('App', frame)

        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    
    return positions, velocities, times

In [ ]:
cap = cv2.VideoCapture(0)
fps = cap.get(cv2.CAP_PROP_FPS)

radius = 3.8 # real tennis ball radius in cm
focal_length = 2.95 * 1280 / 6.35 # focal length of the camera in pixels

lower_green_and_yellow = np.array([25, 52, 72])
upper_green_and_yellow = np.array([102, 255, 255])

positions, velocities, times = position_velocity_sphere(cap, radius, focal_length, fps, upper_green_and_yellow, lower_green_and_yellow)

In [ ]:
# calculate the mean and standard deviation of the velocities and times in groups of N values
N = 8
mean_velocities = []
mean_times = []
err_velocities = []
err_times = []

for i in range(0, len(velocities), N):
    mean_velocities.append(np.mean(velocities[i:i+N]))
    mean_times.append(np.mean(times[i:i+N]))
    err_velocities.append(np.std(velocities[i:i+N]/np.sqrt(N)))
    err_times.append(np.std(times[i:i+N]/np.sqrt(N)))
    
# plot the velocities
import matplotlib.pyplot as plt
plt_, fig = plt.subplots(1, 2, figsize=(15, 5))
fig[0].plot(mean_times, mean_velocities, marker='.', color='red')
fig[0].errorbar(mean_times, mean_velocities, xerr = err_times, yerr=err_velocities, fmt='.', color='black', alpha=0.5)
fig[0].set_xlabel('Time (s)')
fig[0].set_ylabel('Velocity (m/s)')
fig[0].set_title('Velocity vs Time Filtered')
fig[0].grid()
fig[1].plot(times, velocities)
fig[1].set_xlabel('Time (s)')
fig[1].set_ylabel('Velocity (m/s)')
fig[1].set_title('Velocity vs Time Original')
fig[1].grid()

In [ ]:
# select the velocities that are compatible with the pendulum motion

mean_velocities2 = []
mean_times2 = []
err_velocities2 = []
err_times2 = []

for i in range(len(mean_velocities)):
    if mean_times[i] > 3.73 and mean_times[i] < 16:
        mean_velocities2.append(mean_velocities[i])
        mean_times2.append(mean_times[i])
        err_velocities2.append(err_velocities[i])
        err_times2.append(err_times[i])
        
# plot the velocities
import matplotlib.pyplot as plt
plt.plot(mean_times2, mean_velocities2, marker='.', color='red')
plt.errorbar(mean_times2, mean_velocities2, xerr = err_times2, yerr=err_velocities2, fmt='.', color='black', alpha=0.5)
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.title('Velocity vs Time')
plt.grid()

In [ ]:
# fit data with Minuit
from iminuit import Minuit
from iminuit.cost import LeastSquares

l = 1.15 # pendulum length in m
def model(t, g, A):
    return A * np.sqrt(g*l) * abs(np.sin(np.sqrt(g/l) * t))

ls = LeastSquares(mean_times2, mean_velocities2, err_velocities2, model)
m = Minuit(ls, g=9.81, A=np.arctan(np.max(positions)/l))
m.migrad()
m.hesse()
display(m)

# plot the fit
plt.errorbar(mean_times2, mean_velocities2, xerr = err_times2, yerr=err_velocities2, fmt='.', color='black', alpha=0.5)
x = np.linspace(np.min(mean_times2), np.max(mean_times2), 1000)
plt.plot(x, model(x, m.values['g'], m.values['A']), color='blue')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')
plt.title('Velocity vs Time')
plt.grid()
print(f'Velocità massima rilevata: {np.max(mean_velocities2)}')